In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sf-crime/train.csv.zip
/kaggle/input/sf-crime/sampleSubmission.csv.zip
/kaggle/input/sf-crime/test.csv.zip


In [2]:
train = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
test = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')

display(train.head(), test.head())
# 데이터가 거의 90만개 --> 대용량의 데이터셋 처리해주어야 한다
# train/test 겹치지 않는 column 존재한다

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [3]:
alldata = pd.concat([train, test])
alldata

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,NaN
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,NaN
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,NaN
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,NaN
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,NaN
...,...,...,...,...,...,...,...,...,...,...
884257,2003-01-01 00:01:00,NaN,NaN,Wednesday,MISSION,NaN,2600 Block of BRYANT ST,-122.408983,37.751987,884257.0
884258,2003-01-01 00:01:00,NaN,NaN,Wednesday,NORTHERN,NaN,1900 Block of WASHINGTON ST,-122.425342,37.792681,884258.0
884259,2003-01-01 00:01:00,NaN,NaN,Wednesday,INGLESIDE,NaN,5500 Block of MISSION ST,-122.445418,37.712075,884259.0
884260,2003-01-01 00:01:00,NaN,NaN,Wednesday,BAYVIEW,NaN,1500 Block of HUDSON AV,-122.387394,37.739479,884260.0


In [4]:
# TF/IDF term frequency , 빈도수 기준으로 자연어처리
# 하나의 단어 -> 여러가지의 의미로 이루어져 있음 

# 딥러닝 방법 -> 임베딩

In [5]:
alldata['Dates'] = pd.to_datetime(alldata['Dates'])
alldata['Year'] = alldata['Dates'].dt.year # 연도마다 특징, 해당 해의 경기, 연도마다 범죄 category 다름 ex)요새 들어 금융사기 증가, 소매치기 감소 
alldata['Month'] = alldata['Dates'].dt.month # 계절에 따라 범죄 다르게 일어남
alldata['Day'] = alldata['Dates'].dt.day # 특정날짜에 많이 발생하는 범죄 (ex 13일의 금요일, 월말에 생계형 범죄)
alldata['hour'] = alldata['Dates'].dt.hour # 밤에 특정 범죄 많이 일어날수도 

In [6]:
# 정답 column 조회
train['Category'].value_counts() 

LARCENY/THEFT                  174900
OTHER OFFENSES                 126182
NON-CRIMINAL                    92304
ASSAULT                         76876
DRUG/NARCOTIC                   53971
VEHICLE THEFT                   53781
VANDALISM                       44725
WARRANTS                        42214
BURGLARY                        36755
SUSPICIOUS OCC                  31414
MISSING PERSON                  25989
ROBBERY                         23000
FRAUD                           16679
FORGERY/COUNTERFEITING          10609
SECONDARY CODES                  9985
WEAPON LAWS                      8555
PROSTITUTION                     7484
TRESPASS                         7326
STOLEN PROPERTY                  4540
SEX OFFENSES FORCIBLE            4388
DISORDERLY CONDUCT               4320
DRUNKENNESS                      4280
RECOVERED VEHICLE                3138
KIDNAPPING                       2341
DRIVING UNDER THE INFLUENCE      2268
RUNAWAY                          1946
LIQUOR LAWS 

In [7]:
# column 맞춰주기, 날짜 데이터 제거
# 학습 가능한 column 들만 남기기 -> (categorical 변수들 label encoding 필요)
alldata2 = alldata.drop(columns = ['Dates', 'Category', 'Descript', 'Resolution', 'Id'])
alldata2

,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,hour
0,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23
1,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23
2,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13,23
3,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13,23
4,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13,23
...,...,...,...,...,...,...,...,...,...
884257,Wednesday,MISSION,2600 Block of BRYANT ST,-122.408983,37.751987,2003,1,1,0
884258,Wednesday,NORTHERN,1900 Block of WASHINGTON ST,-122.425342,37.792681,2003,1,1,0
884259,Wednesday,INGLESIDE,5500 Block of MISSION ST,-122.445418,37.712075,2003,1,1,0
884260,Wednesday,BAYVIEW,1500 Block of HUDSON AV,-122.387394,37.739479,2003,1,1,0


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # 레이블 인코더 선언
alldata2['DayOfWeek'] = le.fit_transform(alldata2['DayOfWeek']) # 등록 -> 변환 한꺼번에 해준다
alldata2['PdDistrict'] = le.fit_transform(alldata2['PdDistrict'])
alldata2['Address'] = le.fit_transform(alldata2['Address'])
alldata2

,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,hour
0,6,4,20895,-122.425892,37.774599,2015,5,13,23
1,6,4,20895,-122.425892,37.774599,2015,5,13,23
2,6,4,24169,-122.424363,37.800414,2015,5,13,23
3,6,4,4418,-122.426995,37.800873,2015,5,13,23
4,6,5,1923,-122.438738,37.771541,2015,5,13,23
...,...,...,...,...,...,...,...,...,...
884257,6,3,7919,-122.408983,37.751987,2003,1,1,0
884258,6,4,5596,-122.425342,37.792681,2003,1,1,0
884259,6,2,11612,-122.445418,37.712075,2003,1,1,0
884260,6,0,4393,-122.387394,37.739479,2003,1,1,0


In [9]:
# train, test 데이터 분리
train2 = alldata2[:len(train)] 
test2 = alldata2[len(train):]

display(train2, test2)

,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,hour
0,6,4,20895,-122.425892,37.774599,2015,5,13,23
1,6,4,20895,-122.425892,37.774599,2015,5,13,23
2,6,4,24169,-122.424363,37.800414,2015,5,13,23
3,6,4,4418,-122.426995,37.800873,2015,5,13,23
4,6,5,1923,-122.438738,37.771541,2015,5,13,23
...,...,...,...,...,...,...,...,...,...
878044,1,8,16536,-122.459033,37.714056,2003,1,6,0
878045,1,2,11840,-122.447364,37.731948,2003,1,6,0
878046,1,7,11662,-122.403390,37.780266,2003,1,6,0
878047,1,7,23738,-122.390531,37.780607,2003,1,6,0


,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,hour
0,3,0,6626,-122.399588,37.735051,2015,5,10,23
1,3,0,10069,-122.391523,37.732432,2015,5,10,23
2,3,4,6553,-122.426002,37.792212,2015,5,10,23
3,3,2,10985,-122.437394,37.721412,2015,5,10,23
4,3,2,10985,-122.437394,37.721412,2015,5,10,23
...,...,...,...,...,...,...,...,...,...
884257,6,3,7919,-122.408983,37.751987,2003,1,1,0
884258,6,4,5596,-122.425342,37.792681,2003,1,1,0
884259,6,2,11612,-122.445418,37.712075,2003,1,1,0
884260,6,0,4393,-122.387394,37.739479,2003,1,1,0


In [10]:
alldata2.nunique() # Categorical인 Address가 24777개로 종류가 너무 많다

DayOfWeek         7
PdDistrict       10
Address       24777
X             37348
Y             37241
Year             13
Month            12
Day              31
hour             24
dtype: int64

In [11]:
# 모델의 점수 내부적으로 알아야 함 : Validation Data Set 사용
# 회귀 문제: MSE(Mean Squared Error) 주로 사용
# 분류 문제: AUROC, F1Score, LogLoss, ...

# Create Validation Data Set
from sklearn.model_selection import train_test_split

# test_size 기본값: 0.25
# train_test_split 실행할 때 마다 랜덤하게 나옴 -> random_state 사용해 고정해줘야 함(내부적으로 데이터 몇 번 섞는지 지정)
#     -> 데이터들이 계속 다르게 들어가게 되면 전처리를 하게 되었을 때 어떤 영향을 주는지 보다 객관적으로 확인 가능
x_train, x_valid, y_train, y_valid = train_test_split(train2, train['Category'], test_size = 0.2, random_state = 42) 
display(x_train, x_valid)

,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,hour
81381,6,3,4852,-122.419672,37.765050,2014,4,16,6
238545,3,3,8112,-122.408831,37.750390,2012,2,26,10
823641,2,8,5359,-122.475536,37.756153,2003,9,20,19
497355,5,9,17380,-122.415508,37.781654,2008,5,13,13
484193,1,9,21046,-122.412971,37.785788,2008,7,21,18
...,...,...,...,...,...,...,...,...,...
259178,1,9,1161,-122.405895,37.786734,2011,11,14,17
365838,0,7,8614,-122.396759,37.773173,2010,4,9,22
131932,2,3,6587,-122.426956,37.769247,2013,8,10,22
671155,0,0,7608,-122.386942,37.754168,2005,11,4,5


,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,hour
349598,3,9,2556,-122.407878,37.785968,2010,7,4,23
766313,2,5,7480,-122.443597,37.782644,2004,6,26,22
169887,2,7,18135,-122.400474,37.785029,2013,2,9,21
594704,3,4,24171,-122.419698,37.777301,2006,12,3,22
47900,3,5,14154,-122.451488,37.767516,2014,9,21,8
...,...,...,...,...,...,...,...,...,...
65098,1,1,4116,-122.416452,37.794334,2014,7,7,16
336775,3,3,11497,-122.415922,37.759612,2010,9,12,3
30094,1,6,9794,-122.455468,37.786211,2014,12,22,11
3747,1,2,1228,-122.452290,37.725693,2015,4,27,11


In [12]:
# ## Random Forest Classifier
# from sklearn.ensemble import RandomForestClassifier

# rf = RandomForestClassifier(n_jobs = -1) # 모델 선언
# rf.fit(train2, train['Category']) # 모델 학습
# # result = rf.predict(test2) 
# # predict로 하면 점수 잘 나오지 않음 -> submission 확인하기
# # 각 39개 class에 대한 확률로 확인해 입력하기 
# result = rf.predict_proba(test2)
# # 모델 돌아가는지 잘 모르겠고, 운이 안 좋으면 학습이 끝나지 않는다 -> Random Forest 쓸 수 없는 상황 나옴



의사결정 tree 모델 -> 데이터 주어지면 '구분'만 지어줌 ex) 3, 8 들어가면 3과 8이 다르다 학습
Decision Tree 하나만 만들면 발전 X -> 여러 개 만듬

어떤 column을 통해 질문하면 더 데이터가 세세하게 나누어진다 (정보의 이득), column 선택하며 깊어진다 


1. Bagging ex) Random Forest
    - 나무들 만들 때 'Random'하게 만듦
    - 모든 데이터 가져가서 학습시키지 X 
    - 항상 똑같은 나무를 생성해 버리면 나무들이 학습하는 방식 비슷해짐(나무 여러 그루 만들 필요가 없어짐, 비용 증가)
    - '앙상블' 기법, 각각의 학습 방법 다른 나무들 모아서 학습
    - 하나의 나무가 너무 깊게 학습(내부적으로 option-parameter 중 max_depth가 깊어질수록 모델 복잡해짐, default = -1: 무한히 깊어짐, too much 학습)
    - 중요한 column(이 데이터에선는 Address- 24777)을 눈여겨보게 됨 하지만 Address column 너무 카테고리가 많음
    - Address Column 적당히 비슷한 것 끼리 묶어줘야 하는데 다 다른 것으로 인식 -> tree 너무 깊어짐
    - Ram 터짐
2. Boosting ex) XGBoost, LightGBM, Catboost
    - Bagging, Boosting 여러 나무 생성
    - Boosting -> 나무들 서로 '도와'주면서 학습 (Bagging 에서 서로 학습시 도와주지 X)
    - 서로 학습한 내용 볼 수 있음 (like 오답정리)
    - 모델 성능 더 잘 나올 수 있는 가능성 있음 (feedback 받으니까)
    - BUT outlier에 취약, 새롭게 들어오는 데이터셋에 대한 학습X, train 데이터셋에 대한 학습이 좋은 것 => overfitting
    - test set 의 데이터 분포 train과 다른 경우 overfitting
    - hyperparameter 잘 설정해주어야 함
    - boosting -> 내부적으로 수학(사칙연산, 미분) 함 계산기 입장에서는 쉬운 연산들
    - 성능 좋은 CPU 4개 보다 GPU 통해 성능 안 좋은 CPU 여러 개 사용하는것이 학습 속도에 이득
    - Kaggle 에서 GPU 기본으로 제공해 줌(데이터 몇십만개 이상일 때 GPU 쓰는 것이 이득, 적을 때는 오히려 CPU 이득일 수 있음)

In [13]:
# Catboost Classifier
from catboost import CatBoostClassifier

# 모델 선언, task_type: 사용기기 지정
# verbose: 특정 epoch 마다 찍히게 함 (훨씬 깔끔하게 학습 확인 가능)
cbc = CatBoostClassifier(task_type = "GPU", verbose = 20) 
cbc.fit(x_train, y_train, eval_set = (x_valid, y_valid)) # fit with evaluation dataset
# train set 내부의 점수, validation 점수 같이 출력해줌 (본 적 없는 평가set 들어왔을 때 예측 어떻게 하는지)


# n_jobs -> 기본적으로 부스팅 모델 모든 cpu 가져다 씀
# catboost -> boosting모델 중 가장 사용하기 쉬움
# 어느정도 내부적으로 parameter데이터에 맞줘서 설정
# 시간 너무 오래 걸린다 -> 실험을 많이 할 수 없다
# 점수는 비슷하게 유지하되 점수는 유지해야 한다 -> GPU
# 30초 - 1분 GPU 처음 켰을 때 모델 돌릴 때 환경설정 하는 시간 좀 걸림
# GPU 사용시 CPU로 돌릴 때 보다 살짝식 값이 바뀔 수 있음 

Learning rate set to 0.203863
0:	learn: 3.0591920	test: 3.0577220	best: 3.0577220 (0)	total: 80.9ms	remaining: 1m 20s
20:	learn: 2.4719394	test: 2.4730792	best: 2.4730792 (20)	total: 1.48s	remaining: 1m 9s
40:	learn: 2.4348591	test: 2.4388599	best: 2.4388599 (40)	total: 3.37s	remaining: 1m 18s
60:	learn: 2.4130650	test: 2.4205672	best: 2.4205672 (60)	total: 4.71s	remaining: 1m 12s
80:	learn: 2.3965134	test: 2.4070486	best: 2.4070486 (80)	total: 6.05s	remaining: 1m 8s
100:	learn: 2.3837547	test: 2.3975622	best: 2.3975622 (100)	total: 7.4s	remaining: 1m 5s
120:	learn: 2.3727729	test: 2.3905684	best: 2.3905684 (120)	total: 8.74s	remaining: 1m 3s
140:	learn: 2.3639670	test: 2.3854326	best: 2.3854326 (140)	total: 10.4s	remaining: 1m 3s
160:	learn: 2.3556231	test: 2.3805174	best: 2.3805174 (160)	total: 11.7s	remaining: 1m 1s
180:	learn: 2.3480393	test: 2.3768010	best: 2.3768010 (180)	total: 13.1s	remaining: 59.1s
200:	learn: 2.3410316	test: 2.3738205	best: 2.3738205 (200)	total: 14.4s	remain

In [14]:
# 대회 평가방식 확인하기 -> Multiclass Logloss
result = cbc.predict_proba(test2) # predict with probability 
result

array([[3.46700315e-03, 1.62192697e-01, 7.56378973e-06, ...,
        1.48255997e-01, 2.64309617e-02, 2.99306573e-02],
       [1.16814909e-03, 8.86987776e-02, 5.55125356e-06, ...,
        5.53574872e-02, 5.72327429e-02, 2.73206335e-02],
       [4.22770597e-03, 9.69116670e-02, 3.25769883e-06, ...,
        7.69065147e-02, 2.16771786e-02, 6.46967893e-03],
       ...,
       [2.50825449e-03, 8.33625766e-02, 3.81243733e-04, ...,
        7.65566141e-02, 3.83273645e-02, 4.26547311e-03],
       [6.15544896e-03, 4.45146702e-02, 1.35338585e-03, ...,
        2.68345848e-02, 1.72332821e-02, 8.45419628e-03],
       [1.30334207e-03, 2.19889084e-02, 1.56786485e-03, ...,
        2.91530825e-02, 9.38225659e-03, 4.68994416e-04]])

In [15]:
sub = pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')
sub
# Submission columns 알파벳 순서로 되어 있음 -> predict_proba 예측값과 순서 동일

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884257,884257,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
884258,884258,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
884259,884259,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
884260,884260,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
sub.iloc[:, 1:] = result # 첫 번째 column 제외한 나머지 접근
sub

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.003467,0.162193,0.000008,3.579427e-04,0.023644,0.000921,0.004388,0.012706,0.003888,...,0.000006,0.004146,0.000744,0.038274,1.020774e-06,0.006601,0.057373,0.148256,0.026431,0.029931
1,1,0.001168,0.088699,0.000006,2.165320e-04,0.009954,0.002619,0.009103,0.027790,0.005707,...,0.000002,0.002820,0.000107,0.023666,9.429047e-07,0.002429,0.019316,0.055357,0.057233,0.027321
2,2,0.004228,0.096912,0.000003,1.813362e-05,0.108774,0.000687,0.000511,0.004276,0.006397,...,0.000002,0.009351,0.000027,0.032362,7.563814e-08,0.019982,0.070622,0.076907,0.021677,0.006470
3,3,0.001175,0.205524,0.000005,4.955246e-04,0.028433,0.001525,0.003059,0.030091,0.007407,...,0.000018,0.012912,0.000027,0.056210,9.941346e-07,0.002608,0.063663,0.139882,0.046610,0.028294
4,4,0.001175,0.205524,0.000005,4.955246e-04,0.028433,0.001525,0.003059,0.030091,0.007407,...,0.000018,0.012912,0.000027,0.056210,9.941346e-07,0.002608,0.063663,0.139882,0.046610,0.028294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884257,884257,0.005627,0.076809,0.002114,1.704027e-04,0.016285,0.003707,0.000800,0.014086,0.011940,...,0.000959,0.007250,0.000022,0.090178,1.798952e-07,0.015214,0.046778,0.026227,0.018914,0.003045
884258,884258,0.000608,0.040197,0.001993,6.825859e-07,0.068038,0.019487,0.000598,0.007318,0.002859,...,0.000006,0.010636,0.000006,0.085439,1.337472e-07,0.012861,0.041506,0.025204,0.008574,0.000365
884259,884259,0.002508,0.083363,0.000381,5.511482e-04,0.023910,0.014534,0.000014,0.044970,0.003639,...,0.002954,0.008342,0.007014,0.084033,1.634635e-06,0.004957,0.011846,0.076557,0.038327,0.004265
884260,884260,0.006155,0.044515,0.001353,4.581411e-05,0.018074,0.043622,0.000011,0.004326,0.001395,...,0.004092,0.001542,0.000025,0.060744,9.184195e-07,0.005260,0.021612,0.026835,0.017233,0.008454


In [17]:
sub.to_csv('submission', index = False) # --> 파일이 너무 커서 문제 생김 
# 한 번 제출시 시간이 오래 걸림(csv 생성, 다운로드, 업로드, 채점 과정 오래 걸림)
# 진행 중인 대회를 할 때 -> 하루에 제출 횟수 정해져 있음
# 모델의 점수 내부적으로 알아야 함 : Validation Data Set 사용
# 회귀 문제: MSE(Mean Squared Error) 주로 사용
# 분류 문제: AUROC, F1Score, LogLoss, ...